<a href="https://colab.research.google.com/github/tajuar-akash-hub/test_repo_for_colaboration/blob/main/Final_notebooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problems :
1) In the AdaBoost cell during "Grid Search" it is taking a huge time. <br>
2) All "Grid search" cells taking huge time to execute.

In [ ]:
!git clone https://github.com/tajuar-akash-hub/Datasets.git

In [ ]:
import pandas as pd
import numpy as np

insurance_fraud=pd.read_csv('/content/Datasets/Health Insurance Fraud Claims csv file.csv')

insurance_fraud.head(2)

In [ ]:
insurance_fraud.describe()

In [ ]:
age_under_18=insurance_fraud[insurance_fraud['PatientAge']< 18]
len(age_under_18)

In [ ]:
insurance_fraud = insurance_fraud.drop(age_under_18.index)
insurance_fraud.shape

#remove unwanted columns from the datasets

In [ ]:
insurance_fraud = insurance_fraud[["ClaimDate","ClaimAmount","PatientAge","PatientGender","ProviderSpecialty","ClaimStatus","PatientIncome","PatientMaritalStatus","PatientEmploymentStatus","ProviderLocation","ClaimType","ClaimSubmissionMethod","ClaimLegitimacy"]]

print(f"Shape of the datasets: {insurance_fraud.shape}")

insurance_fraud.head(1)

# all coulmns unique values

In [ ]:
all_unique=insurance_fraud.nunique()
all_unique_df = pd.DataFrame(all_unique, columns=['Unique Value Count'])
all_unique_df

# Check duplicated values

In [ ]:
duplicates=insurance_fraud.duplicated()
flag=0
for duplicate in duplicates:
  if duplicate == True:
    flag=1
    print(f"duplicate values {duplicate}")
if flag == 0:
  print("No duplicates row")


# Check Outliers with  IQR outliers data

In [ ]:
iqr_outlier_data = insurance_fraud.copy()


In [ ]:
def iqr_outliers(data,x):
  Q1=np.percentile(data[x],25)
  Q3=np.percentile(data[x],75)
  IQR=Q3-Q1

  lower_bound=Q1 - (1.5 * IQR)
  upper_bound=Q1 + (1.5 * IQR)

  outliers=[]
  outliers_index=[]
  for i,value in data[x].items():
    if value< lower_bound or value > upper_bound:
      outliers.append(value)
      outliers_index.append(i)
  return outliers,outliers_index

In [ ]:
## outliers in PatientIncome
income,outliers_index=iqr_outliers(iqr_outlier_data,"PatientIncome")


print(f"outliers in PatientIncome : {len(income)}")

print(f"outliers Indexes in PatientIncome : {outliers_index}")

In [ ]:
outliers_remove_data = iqr_outlier_data.copy()

outliers_replaced_data = iqr_outlier_data.copy()  #not using it currently

#ques : which on to use ?

print(outliers_remove_data.shape)
print(outliers_remove_data.shape)

In [ ]:
outliers_remove_data = outliers_remove_data.drop(index=outliers_index)

#here replace the outlier with mean income


print(f"After removing outliers from 'PatientIncome': {outliers_remove_data.shape}")

## Replace PatientAge columns with mean_age

In [ ]:

# 1. Detect and replace outliers in 'PatientAge'

_,outliers_index=iqr_outliers(outliers_remove_data,"PatientIncome")

mean_income = np.mean(outliers_remove_data['PatientAge'])

mean_age = int(mean_income)

# Replace the outliers with the mean in 'PatientAge'

for index in outliers_index:
    outliers_remove_data.loc[index, 'PatientAge'] = mean_age

#Encoding Categorical values

In [ ]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
le=LabelEncoder()

# Use Ordinal encoders to encode ClaimStatus

In [ ]:
ordinal_categories=["Denied","Pending","Approved"]

ordinal=OrdinalEncoder(categories=[ordinal_categories])

outliers_remove_data["ClaimStatus"]=ordinal.fit_transform(outliers_remove_data[["ClaimStatus"]])

outliers_remove_data.head()

In [ ]:
categorical_columns=["PatientGender",
                     "ClaimLegitimacy",
                     "ProviderSpecialty",
                     "PatientMaritalStatus",
                     "PatientEmploymentStatus",
                     "ProviderLocation",
                     "ClaimType",
                     "ClaimSubmissionMethod"
                     ]

for col in categorical_columns:
  outliers_remove_data[col]=le.fit_transform(outliers_remove_data[col])

outliers_remove_data.head()

In [ ]:
outliers_remove_data.info()

# Check ClaimLegitimacy distributions

In [ ]:
value_counts = outliers_remove_data.groupby('ClaimLegitimacy').size().reset_index(name = 'Count')

value_counts

# OverSampling

In [ ]:
# Convert 'ClaimDate' to ordinal
outliers_remove_data['ClaimDate'] = pd.to_datetime(outliers_remove_data['ClaimDate']).apply(lambda date: date.toordinal())

from imblearn.over_sampling import SMOTE

X_smote = outliers_remove_data.drop('ClaimLegitimacy',axis=1)
y_smote = outliers_remove_data['ClaimLegitimacy']

smote = SMOTE(random_state=42)

X_resampled_replace , y_resampled_replace = smote.fit_resample(X_smote,y_smote)

outliers_remove_data = pd.DataFrame(X_resampled_replace , columns = X_smote.columns)

outliers_remove_data['ClaimLegitimacy'] = y_resampled_replace

In [ ]:
len(X_resampled_replace)

In [ ]:
len(y_resampled_replace)

In [ ]:
len(outliers_remove_data)

In [ ]:
value_counts =outliers_remove_data.groupby('ClaimLegitimacy').size().reset_index(name = 'Count')

value_counts

# Data Spliting (Target and features variables) with Oversampling in outliers_replace_data

In [ ]:
#import libraries

from sklearn.model_selection import train_test_split,KFold,cross_val_score

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
min_max=MinMaxScaler()

X = outliers_remove_data.drop('ClaimLegitimacy',axis=1)
y = outliers_remove_data['ClaimLegitimacy']

X_smote_scaled_replace = min_max.fit_transform(X)



X_train,X_test,y_train,y_test = train_test_split(X_smote_scaled_replace,y,test_size=0.2,random_state = 42)


# why after scaling X size reduced  (initially it was 6862, now 5489)


In [ ]:
len(X_smote_scaled_replace)

In [ ]:
len(X_train)

In [ ]:
X_train.shape

# **Start applying ML Models**

# LogisticRegressionClassifiers

In [ ]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg_replace = LogisticRegression(random_state=16)

# fit the model with data

logreg_replace.fit(X_train,y_train)

#predict the outputs
logreg_ypreds_replace= logreg_replace.predict(X_test)

In [ ]:
# Evaluate model accuracy

accuracy_logreg_replace = accuracy_score(y_test, logreg_ypreds_replace)
print(f'Accuracy with smote data: {accuracy_logreg_replace * 100:.2f}%')

# Print classification report
print("\nClassification Report with smote data:")

print(classification_report(y_test, logreg_ypreds_replace))

# Print confusion matrix
print("\nConfusion Matrix with smote data:")

print(confusion_matrix(y_test, logreg_ypreds_replace))

#  Ada Boost Classifiers

In [ ]:
# Create adaboost classifer object
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

#Instanntiate the model
replace_adaboost = AdaBoostClassifier()

param_grid = {
    'n_estimators': [50, 100, 150,200],        # Test different numbers of estimators
    'learning_rate': [0.01, 0.1, 0.5, 1],  # Test different learning rates
    'estimator': [DecisionTreeClassifier(max_depth=1),  # Shallow tree
                       DecisionTreeClassifier(max_depth=3)]  # Slightly deeper tree
}


In [ ]:
# Create a GridSearchCV object with cross-validation

grid_search = GridSearchCV(estimator=replace_adaboost,
                           param_grid=param_grid,
                           cv=5,  # 5-fold cross-validation
                           scoring='accuracy',  # Evaluate using accuracy
                           n_jobs=-1)  # Use all available cores

# Fit the model with grid search on the training data

grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score

print(f"Best Parameters: {grid_search.best_params_}")

print(f"Best Score: {grid_search.best_score_}")

In [ ]:
# Use the best model to predict on the test set
best_model = grid_search.best_estimator_
y_pred_ada = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_acc_ada_replace = accuracy_score(y_test, y_pred_ada)

# Print the test set accuracy
print(f'Accuracy with smote data: {test_acc_ada_replace * 100:.2f}%')

# Print classification report
print("\nClassification Report with smote data:")
print(classification_report(y_test, y_pred_ada))

# Print confusion matrix
print("\nConfusion Matrix with smote data:")
print(confusion_matrix(y_test, y_pred_ada))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(7,5))
sn.heatmap(confusion_matrix(y_test, y_pred_ada), annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Truth')

# **ExraTreeClassifiers**

In [ ]:

from sklearn.ensemble import ExtraTreesClassifier

# Define the ExtraTreesClassifier
extra_tree_replace = ExtraTreesClassifier(random_state=42)

# Define a parameter grid for tuning

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}





In [ ]:
extraclassi_model=ExtraTreesClassifier()

extraclassi_model.fit(X_train,y_train)


In [ ]:
importance_features=pd.Series(extraclassi_model.feature_importances_,index=X.columns)
importance_features

In [ ]:
importance_features.nlargest(10).plot(kind='bar',color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

In [ ]:

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=extra_tree_replace, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model with grid search on the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score from the grid search
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_}")

# Use the best model to predict on the test set
best_model = grid_search.best_estimator_
y_pred_extra = best_model.predict(X_test)

# Calculate accuracy on the test set
test_acc_extratree_replace = accuracy_score(y_test,y_pred_extra)
print(f"Test Accuracy: {test_acc_extratree_replace}")

# Voting Classifiers

In [ ]:
# Import necessary libraries
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


log_clf = LogisticRegression(random_state=42)
rf_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(probability=True, random_state=42)


# Soft

In [ ]:
# Create a Voting Classifier with soft voting
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rf_clf), ('svc', svm_clf)],
    voting='soft'  # Use 'hard' for majority voting
)

# Train the Voting Classifier
voting_clf.fit(X_train, y_train)

# Make predictions
y_pred = voting_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Voting Classifier Accuracy: {accuracy:.4f}")

# Evaluate individual models
for clf in (log_clf, rf_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred_individual = clf.predict(X_test)
    print(f"{clf.__class__.__name__} Accuracy: {accuracy_score(y_test, y_pred_individual):.4f}")


# Hard

In [ ]:
# Create a Voting Classifier with soft voting

voting_clf_hard = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rf_clf), ('svc', svm_clf)],
    voting='hard'  # Use 'hard' for majority voting
)

# Train the Voting Classifier
voting_clf_hard.fit(X_train, y_train)

# Make predictions
y_pred_hard = voting_clf.predict(X_test)

# Calculate accuracy
accuracy_hard = accuracy_score(y_test, y_pred_hard)
print(f"Voting Classifier Accuracy: {accuracy_hard:.4f}")

# Evaluate individual models
for clf in (log_clf, rf_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred_individual_hard = clf.predict(X_test)
    print(f"{clf.__class__.__name__} Accuracy: {accuracy_score(y_test, y_pred_individual_hard):.4f}")


# SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC()

# Train the model
svc.fit(X_train, y_train)

# Make predictions
y_pred = svc.predict(X_test)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


# Print confusion matrix

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(5,5))
sn.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
X_train

# Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_model = RandomForestClassifier(random_state=42)

# random_model.fit(X_train,y_train)

# Train the model
random_model.fit(X_train , y_train)

# Make predictions
y_pred = random_model.predict(X_test)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(7,5))
sn.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Truth')



#Cross Validation


from sklearn.model_selection import cross_val_score
scores = cross_val_score(random_model, X_train, y_train, cv=5)    #By Akash (experimental)


print(f'Cross-validation scores: {scores}')
print(f'Mean accuracy: {scores.mean() * 100:.2f}%')

# Support vector Classifier

In [ ]:
from sklearn.svm import SVC

svc = SVC()

# Train the model
svc.fit(X_train, y_train)

# Make predictions
y_pred = svc.predict(X_test)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(5,5))
sn.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Truth')

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')                                        #do we need to Scale our data for KNN ( by Akash) ?

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(5,5))
sn.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Truth')

# Logistic Regression ( BY Akash)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty = 'l2' , tol = 0.0001 , solver = 'liblinear',
                           max_iter=100,multi_class='auto')

#tol = tolerance , max_itr = max iteration

model.fit(X_train,y_train)

pred_values = model.predict(X_test)


model.fit(X_train,y_train)


pred_values = model.predict(X_test)


from sklearn.metrics import confusion_matrix

# Get predictions (class labels)
pred_values = model.predict(X_test)

# Compute the confusion matrix
cm = confusion_matrix(y_test, pred_values)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)


#Evaluation metrices

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


accuracy = accuracy_score(y_test,pred_values)

precision = precision_score(y_test,pred_values)

recall = recall_score(y_test,pred_values)

f1 = f1_score(y_test,pred_values)

AUC = roc_auc_score(y_test,pred_values)


print("accuracy : ",accuracy*100)
print("precision : ",precision*100)
print("recall : ",recall*100)
print("f1 score : ",f1*100)
print("AUC Score : ",AUC*100)



#Roc curve


# Get predicted probabilities
y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probability for the positive class (1)

# Calculate fpr and tpr for the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Plot the ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], ls="--")  # Random guess line

plt.title("ROC Curve for Logistic Regression")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

# Naive Bayes Classifier (By Akash)

In [ ]:
from sklearn.preprocessing import StandardScaler

#Standard Scaler for Naive Bayes

STD_Scaler = StandardScaler()
STD_Scaled_X =  STD_Scaler.fit_transform(X)
STD_Scaled_X


#train-test split

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(STD_Scaled_X,y,test_size=0.20)

print("train data size (features):",len(x_train))
print("train data size (target):",len(y_train))

print("test data size(feature)",len(x_test))
print("test data size(target)",len(y_test))


len(X_train)

In [ ]:
#K-fold validation

from sklearn.naive_bayes import GaussianNB

NB_Classifier = GaussianNB(priors= None , var_smoothing =  1e-09 )    #1e-09 = 1*10^-9


#we add very small value in var_smoothing to avoid 0/0 form

# priors = prior probability  (see the documentation from sklearn)

# var_smoothing = variance smothing where variance = sigma square



In [ ]:
#K fold for hyper parameter tuning

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

K_fold = KFold(10)

accuracy = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'accuracy' )

precision = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'precision' )

recall = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'recall' )

f1_score = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'f1' )

AUC = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'roc_auc' )



In [ ]:
accuracy
overall_accuracy = sum(accuracy) / len(accuracy)
print(f'Overall Accuracy = {overall_accuracy*100:0.2f}%')


recall
overall_recall = sum(recall) / len(recall)
print(f'overall_recall = {overall_recall*100:0.2f}%')


f1_score

overall_f1_score = sum(f1_score) / len(f1_score)
print(f'overall_f1_score = {overall_f1_score*100:0.2f}%')


overall_AUC = sum(AUC) / len(AUC)
print(f'overall_AUC = {overall_AUC*100:0.2f}%')

In [ ]:
#Applying the Algorithm after k-fold

NB_Classifier = GaussianNB(priors= None , var_smoothing =  1e-09 )  #tune var_smoothing
NB_Classifier.fit(X_train,y_train)

#unseen data

unseen_prediction = NB_Classifier.predict(X_test)

In [ ]:
#Evaluation metrices

from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score ,roc_curve ,roc_auc_score

accuracy = accuracy_score(unseen_prediction,y_test)

precision = precision_score(unseen_prediction,y_test)

recall = recall_score(unseen_prediction,y_test)

f1 = f1_score(unseen_prediction,y_test)

AUC = roc_auc_score(unseen_prediction,y_test)

print(f'accuracy = {accuracy*100:0.2f}%')
print(f'precision = {precision*100:0.2f}%')
print(f'recall = {recall*100:0.2f}%')
print(f'f1 = {f1*100:0.2f}%')

# Decision Tree Classifier (Akash)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DTregressor = DecisionTreeRegressor(criterion='squared_error',
                                  max_depth= 10,
                                  min_samples_split= 2,
                                  min_samples_leaf= 1,
                                  max_features= None,
                                  max_leaf_nodes= None,
                                  min_impurity_decrease= 0.01
                                  )

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

k_fold = KFold(10)

results1 = cross_val_score(DTregressor, X_train, y_train, cv=k_fold , scoring='neg_mean_squared_error')
results2 = cross_val_score(DTregressor, X_train, y_train, cv=k_fold , scoring= 'neg_mean_absolute_percentage_error')
results3 = cross_val_score(DTregressor, X_train, y_train, cv=k_fold , scoring= 'r2')

In [ ]:
results1

In [ ]:
results2

In [ ]:
overall_MAE_percentage = sum(results2)/len(results2)

print(abs(overall_MAE_percentage)*100)

In [ ]:
overallr2 = sum(results3)/len(results3)
print(overallr2)

In [ ]:
print("akash ")